# PageRank Algorithm

In [1]:
function pagerank(M, max_iterations, d) 
    """PageRank: Rank the input pages
    
    Parameters
    ---------- 
    M : Array
        Transition Matrix of Directed Graph of Websites
    
    max_iterations: int
        Maximum iterations for PageRank
    
    d: float (0 < d < 1)
        damping coefficient. The probability that a random surfer will not jump to an arbitrary node
    
    Returns
    -------
    Vector (size N)
        The ranking (0 < ranking < 1) of each webpage
    """
    # Define Margin of Error
    e = 1.0e-6
    
    # Define number of web pages
    N = size(M)[1]
    
    # Evenly distributing random surfer
    v = fill(1.0/N, N)
    
    # Create copy of last v to margin of error
    v_last = copy(v)
    
    # Remove zero columns from transition matrix (replace with 1/N)
    for col in 1:N
        if (M[:,col] == zeros(N))
            M[:,col] = fill(1.0/N, N)
        end
    end
    
    for loop in 1:max_iterations
        # Iterate on v
        v = d * M * v .+ (1 - d) / N
        
        # Calculate the error between current and last state vectors
        difference = abs.(v_last - v) # Sum all error values
        error = sum(difference)
        
        # Compare to margin of error
        if(error < N * e)
            return v
        end

        # Set last state vector to current state
        v_last = v
    end
    
    return v
end

pagerank (generic function with 1 method)

# Example Usage

In [2]:
function pre_process_webpages(webpages)
    """Pre-process Webpages: Define an N x N matrix from webpages data
    
    Parameters 
    ---------- 
    webpages : Array
        A 2D array describing webpages and any existsing links to other webpages
    
    Returns 
    -------
    Matrix (N x N)
        The corresponding transition matrix to the input webpage array
    """
    # Get N: size of matrix
    N = size(webpages)[1]
    
    # Define M: an empty N x N matrix with values 0
    M = zeros((N, N))
    
    # For faster lookup processing, reindex websites from naturals to
        # the websites themselves
        # Define lookup dictionary
    lookup_dict = Dict{String,Int64}()
        # Load lookup dictionary
    
    for website in 1:N 
        lookup_dict[webpages[website][1]] = website;
    end
        
    # Insert each website into the
    for website in 1:N
            # Get number of valid links
            # Needs to loop through instead of calling size()
            # because if link not defined in webpages we discard it
        number_of_edges = 0
        for link in webpages[website][2]
            # Check that valid link and link is not the web page itself
            if (haskey(lookup_dict, link) && webpages[website][1] != link)
                number_of_edges = number_of_edges + 1
            end
        end
        
        # Write the probability of moving from node j to i # for each M_{ij}
        for link in webpages[website][2]
            if (haskey(lookup_dict, link) && webpages[website][1] != link)
                M[lookup_dict[link], website] = 1/number_of_edges
            end 
        end
    end
    return M 
end

pre_process_webpages (generic function with 1 method)

In [3]:
function print_output(webpages, result)
    """Print Output: Rank and output the ranks in plan text
    
    Parameters
    ---------- 
    
    webpages : Array
        A 2D array describing webpages and any existsing links to other webpages
    
    result: Vector (size N)
        The PageRank (0 < ranking < 1) of each webpage
    
    Returns
    -------
        None
    
    Side Effects
    ------------
    Prints out each website in plain text in order of ranking
    """
    
    # Set the ranks of each result
    ranked_result = sortperm(result, rev=true) # Initialise counter to print rankings
    counter = 1
    
    # Print out rankings
    for webpage_index in 1:size(result)[1] # Get index of this rank:
        index = findall(x->x==webpage_index, ranked_result)[1]

        # Print the web page
        print(string(counter), ". ", webpages[index][1], "\n")
        counter = counter + 1
    end
end

print_output (generic function with 1 method)

In [4]:
DATA = [
    ["https://one.com", ["https://two.com"]], 
    ["https://two.com", ["https://three.com"]], 
    ["https://three.com", ["https://two.com", "https://four.com", "https://one.com"]],
    ["https://four.com", ["https://two.com", "https://three.com"]]
]

4-element Array{Array{Any,1},1}:
 ["https://one.com", ["https://two.com"]]
 ["https://two.com", ["https://three.com"]]
 ["https://three.com", ["https://two.com", "https://four.com", "https://one.com"]]
 ["https://four.com", ["https://two.com", "https://three.com"]]

In [5]:
M = pre_process_webpages(DATA)

4×4 Array{Float64,2}:
 0.0  0.0  0.333333  0.0
 1.0  0.0  0.333333  0.5
 0.0  1.0  0.0       0.5
 0.0  0.0  0.333333  0.0

In [6]:
result = pagerank(M, 100, 0.85)

4-element Array{Float64,1}:
 0.14509120716073326
 0.3300833885110279
 0.37973419716750556
 0.14509120716073326

In [7]:
print_output(DATA, result)

1. https://three.com
2. https://two.com
3. https://one.com
4. https://four.com
